In [28]:
import boto3
import pandas as pd
import io
import datetime
import psycopg2
from io import StringIO, BytesIO 
from twilio.rest import Client

import q_credentials.db_secmaster_cred as db_secmaster_cred
import q_credentials.db_indicator_cred as db_indicator_cred
conn_indicator = psycopg2.connect(host=db_indicator_cred.dbHost , database=db_indicator_cred.dbName, user=db_indicator_cred.dbUser, password=db_indicator_cred.dbPWD)
conn_secmaster = psycopg2.connect(host=db_secmaster_cred.dbHost , database=db_secmaster_cred.dbName, user=db_secmaster_cred.dbUser, password=db_secmaster_cred.dbPWD)

import json


In [38]:
import oandapyV20.endpoints.pricing as pricing
import oandapyV20.endpoints.accounts as accounts
import q_credentials.oanda_cred as oanda_cred
import oandapyV20


In [13]:
s3 = boto3.client('s3', endpoint_url="http://minio-image:9000",aws_access_key_id="minio-image",aws_secret_access_key="minio-image-pass")
Bucket="airflow-files"
Key="financing_cost.csv"
read_file = s3.get_object(Bucket=Bucket, Key=Key)
df = pd.read_csv(read_file['Body'],sep=',')
df

,Instrument,Long rate,Short rate,Long cost,Short cost
0,AUD_CAD,-0.0115,-0.0104,-2.19,-1.98
1,AUD_CHF,-0.0027,-0.0195,-0.51,-3.71
2,AUD_HKD,-0.0193,-0.0169,-3.68,-3.22
3,AUD_JPY,-0.0090,-0.0134,-1.71,-2.55
4,AUD_NZD,-0.0124,-0.0103,-2.36,-1.96
...,...,...,...,...,...
65,USD_SGD,-0.0131,-0.0160,-3.59,-4.38
66,USD_THB,-0.0818,-0.0701,-22.41,-19.21
67,USD_TRY,-0.2343,0.0315,-64.19,8.63
68,USD_ZAR,-0.0655,0.0155,-17.95,4.25


## Direct Rates

In [3]:
margin=100
risk_perc=0.05
symbol="EUR_USD"
entry_price=1.0005
exit_price=1.0003
leverage=50

In [4]:
loss=margin*risk_perc
quantity=loss/(entry_price-exit_price)
margin_required=entry_price*quantity/leverage
margin_required

500.2500000000551

## Indirect Rates

In [5]:
margin=100
risk_perc=0.05
symbol="USD_JPY"
entry_price=120.50
exit_price=120.30
leverage=50

In [6]:
# loss=margin*risk_perc
loss = 166.20
tick_size=0.01
pip = (entry_price-exit_price) / tick_size
print(pip)
pip_value = loss / pip
print(pip_value)
quantity = pip_value * entry_price / tick_size
quantity

20.000000000000284
8.309999999999881


100135.49999999857

In [58]:
# loss=margin*risk_perc
loss = 166.20
tick_size=0.01
pip = (entry_price-exit_price)
print(pip)
pip_value = loss / pip
print(pip_value)
quantity = pip_value * entry_price
quantity

0.20000000000000284
830.9999999999882


100135.49999999857

## Cross Rates

In [60]:
margin=100
risk_perc=0.05
symbol="EUR_GBP"
entry_price=.6760
exit_price=.6750
leverage=50

In [63]:
# loss=margin*risk_perc
loss = 170.54
tick_size=0.01
base_rate=1.1840
pip = (entry_price-exit_price)
print(pip)
pip_value = loss / (pip * base_rate)
print(pip_value)
quantity = pip_value * entry_price 
quantity

0.0010000000000000009
144037.16216216204


97369.12162162155

In [194]:
# Logic from 
# http://www.fxtrademaker.com/fx_calculation.htm
# Checked with 
# https://www.forextime.com/trading-tools/trading-calculator/profit-calculator
# Margins and PnL checked on the oanda web trading platform - do the same before placing orders
   
def oanda_data(instruments):
    client = oandapyV20.API(access_token=oanda_cred.token_practice)
    
    r = pricing.PricingInfo(accountID=oanda_cred.acc_id_practice,params=instruments)
    rv = client.request(r)
    df_price=pd.read_json(json.dumps(rv, indent=2))
    df_price=pd.io.json.json_normalize(data=df_price['prices'])
    df_price=df_price[['instrument','closeoutBid','closeoutAsk']]
    df_price.set_index('instrument',drop=True,inplace=True)
    
    r = accounts.AccountInstruments(accountID=oanda_cred.acc_id_practice,params=instruments)
    rv = client.request(r)
    df_info=pd.read_json(json.dumps(rv, indent=2))
    df_info=pd.io.json.json_normalize(data=df_info['instruments'])
    df_info=df_info[['name','pipLocation','financing.longRate','financing.shortRate','marginRate']]
    df_info.set_index('name',drop=True,inplace=True)
    
    return pd.concat([df_price, df_info], axis=1, sort=False)



def risk_calculator(symbol,entry_price,sl_price,purchase_currency="USD",free_margin=100,risk_perc=0.05,sep="_"):
    max_loss=free_margin*risk_perc
    type="BUY" if entry_price>exit_price else "SELL"
    bid_or_ask = "closeoutAsk" if type=="BUY" else "closeoutBid"
    price_diff = entry_price - sl_price

    price_multiplier = 1 # value in USD per 1 quantity

    symbol_ls=symbol.split(sep)
    if purchase_currency not in symbol_ls: #EUR_GBP
        try:
            base_currency=symbol_ls[0]+sep+purchase_currency #EUR_USD
            params={'instruments':','.join([base_currency,symbol])}
            df = oanda_data(params)
        except:
            base_currency=purchase+sep+symbol_ls[0] #USD_EUR which is wrong 
            params={'instruments':','.join([base_currency,symbol])}
            print(params)
            df = oanda_data(params)

        if base_currency[-3:]==purchase_currency:
            price_multiplier = float(df.loc[base_currency][bid_or_ask])/float(df.loc[symbol][bid_or_ask])
        else:
            price_multiplier = float(df.loc[base_currency][bid_or_ask]) * float(df.loc[symbol][bid_or_ask])

    else:
        params={'instruments':symbol}
        df = oanda_data(params)
        if purchase_currency == symbol_ls[0]:
            price_multiplier = 1/ float(df.loc[symbol][bid_or_ask]) 

            
    
    quantity=max_loss/(price_diff*price_multiplier)
    loss_per_std_lot=price_multiplier * price_diff * 100000
    position_value = price_multiplier * entry_price * quantity
    margin_requirement = position_value*float(df.loc[symbol]['marginRate'])
    margin_for_std_lot=price_multiplier * entry_price * 100000*float(df.loc[symbol]['marginRate'])
    financing_rate = float(df.loc[symbol]['financing.longRate']) if type=="BUY" else float(df.loc[symbol]['financing.shortRate'])
    financing_cost = position_value * financing_rate / 365  
    print("Quantity =",quantity)
    print("Position Value=",position_value)
    print("Financing Cost=",financing_cost)
    print("Margin Requirement",margin_requirement)
    print(max_loss,price_multiplier,price_diff)
    display(df)

In [198]:
free_margin=100
risk_perc=0.05

entry_price=0.91226
sl_price=0.91800
purchase_currency="USD"
symbol="USD_CHF"

risk_calculator(symbol=symbol,entry_price=entry_price,sl_price=sl_price,free_margin=free_margin,risk_perc=risk_perc)


Quantity = -794.6515679442399
Position Value= -794.6515679442399
Financing Cost= 0.043542551668177526
Margin Requirement -31.786062717769596
5.0 1.0961787209786684 -0.005740000000000078


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:13: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  del sys.path[0]
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:20: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


,closeoutBid,closeoutAsk,pipLocation,financing.longRate,financing.shortRate,marginRate
USD_CHF,0.91226,0.91276,-4,-0.00040,-0.02,0.04


In [195]:
free_margin=100
risk_perc=0.05

entry_price=1.17835
sl_price=1.19500
purchase_currency="USD"
symbol="EUR_USD"

risk_calculator(symbol=symbol,entry_price=entry_price,sl_price=sl_price,free_margin=free_margin,risk_perc=risk_perc)


Quantity = -300.30030030029934
Position Value= -353.85885885885773
Financing Cost= 0.0031023242420502597
Margin Requirement -7.0771771771771546
5.0 1 -0.016650000000000054


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:13: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  del sys.path[0]
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:20: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


,closeoutBid,closeoutAsk,pipLocation,financing.longRate,financing.shortRate,marginRate
EUR_USD,1.17835,1.17886,-4,-0.018,-0.0032,0.02


In [196]:
free_margin=100
risk_perc=0.05

entry_price=0.90273
sl_price=0.89783
purchase_currency="USD"
symbol="EUR_GBP"

risk_calculator(symbol=symbol,entry_price=entry_price,sl_price=sl_price,free_margin=free_margin,risk_perc=risk_perc)


Quantity = 781.7312863109322
Position Value= 921.1530612244869
Financing Cost= -0.011861422980150929
Margin Requirement 46.05765306122435
5.0 1.305318312230678 0.0049000000000000155


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:13: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  del sys.path[0]
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:20: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


,closeoutBid,closeoutAsk,pipLocation,financing.longRate,financing.shortRate,marginRate
EUR_USD,1.17835,1.17886,-4,-0.018,-0.0032,0.02
EUR_GBP,0.90273,0.90334,-4,-0.0173,-0.0047,0.05


In [197]:
free_margin=100
risk_perc=0.05

entry_price=105.908
sl_price=104.718
purchase_currency="USD"
symbol="USD_JPY"

risk_calculator(symbol=symbol,entry_price=entry_price,sl_price=sl_price,free_margin=free_margin,risk_perc=risk_perc)


Quantity = 445.1554621848748
Position Value= 444.99159663865635
Financing Cost= 0.0
Margin Requirement 17.799663865546254
5.0 0.00943868160495342 1.1899999999999977


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:13: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  del sys.path[0]
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:20: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


,closeoutBid,closeoutAsk,pipLocation,financing.longRate,financing.shortRate,marginRate
USD_JPY,105.908,105.947,-2,0,0,0.04
